In [98]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [99]:
data = pd.read_csv('movie_bd_v5.xls')
#data.sample(5)

In [11]:
#data.describe()

# Предобработка

In [100]:
def split_values(column): 
    """Splits values in column by separator and returns Series"""
    return pd.Series(column.str.cat(sep='|').split('|')).value_counts()


answers = {} # create a dictionary for answers
months = pd.Series(['January ', 'February', 'March ', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], index=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'])
cast = split_values(data['cast'])
companies = split_values(data['production_companies'])
directors = split_values(data['director'])

# change format in datetime
data.loc[:,'release_date'] = pd.to_datetime(data['release_date'])

"""add new columns"""
data.loc[:, 'release_month'] = data['release_date'].map(lambda x : months[str(x).split('-')[1]])
data.loc[:,'profit'] = data['revenue'] - data['budget']

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [101]:
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650) '

In [27]:
data[data.budget == data.budget.max()][['imdb_id', 'original_title']]

,imdb_id,original_title
723,tt1298650,Pirates of the Caribbean: On Stranger Tides


ВАРИАНТ 2

In [106]:
data.sort_values(by='budget')[['imdb_id', 'original_title']].tail(1)

,imdb_id,original_title
723,tt1298650,Pirates of the Caribbean: On Stranger Tides


# 2. Какой из фильмов самый длительный (в минутах)?

In [102]:
answers['2'] = 'Gods and Generals (tt0279111)'

In [29]:
data.sort_values(by='runtime')[['imdb_id','original_title']].tail(1)

,imdb_id,original_title
1157,tt0279111,Gods and Generals


# 3. Какой из фильмов самый короткий (в минутах)?





In [103]:
answers['3'] = 'Winnie the Pooh (tt1449283)'

In [110]:
data.sort_values(by='runtime')[['imdb_id','original_title', 'runtime']].head(1)

,imdb_id,original_title,runtime
768,tt1449283,Winnie the Pooh,63


# 4. Какова средняя длительность фильмов?


In [104]:
answers['4'] = '110'

In [19]:
round(data['runtime'].mean())

110

# 5. Каково медианное значение длительности фильмов? 

In [105]:
answers['5'] = '107'

In [21]:
round(data['runtime'].median())

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [106]:
answers['6'] = 'Avatar (tt0499549)'

In [112]:
data.sort_values(by='profit')[['imdb_id', 'original_title', 'profit']].tail(1)

,imdb_id,original_title,profit
239,tt0499549,Avatar,2544505847


# 7. Какой фильм самый убыточный? 

In [107]:
answers['7'] = 'The Lone Ranger (tt1210819)'

In [113]:
data.sort_values(by='profit')[['imdb_id', 'original_title', 'profit']].head(1)

,imdb_id,original_title,profit
1245,tt1210819,The Lone Ranger,-165710090


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [108]:
answers['8'] = '1478'

In [115]:
len(data.query('revenue > budget'))

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [109]:
answers['9'] = 'The Dark Knight (tt0468569)'

In [121]:
data.query('release_year == 2008').sort_values(by='revenue')[['imdb_id','original_title', 'revenue']].tail(1)

,imdb_id,original_title,revenue
599,tt0468569,The Dark Knight,1001921825


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [110]:
answers['10'] = 'The Lone Ranger (tt1210819) '

In [128]:
data.query('2012 <= release_year <= 2014').sort_values(by='profit')[['imdb_id', 'original_title', 'profit']].head(1)

,imdb_id,original_title,profit
1245,tt1210819,The Lone Ranger,-165710090


# 11. Какого жанра фильмов больше всего?

In [111]:
answers['11'] = 'Drama'

In [137]:
split_values(data.genres).value_counts().head(1)

Drama    782
dtype: int64

ВАРИАНТ 2

In [164]:
genres = data['genres'].map(lambda x : x.split('|')).sum()
Counter(genres).most_common(1)

[('Drama', 782)]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [112]:
answers['12'] = 'Drama'

In [153]:
df = split_values( data.query('profit > 0')['genres'] )
df.value_counts().head(1)

Drama    560
dtype: int64

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [113]:
answers['13'] = 'Peter Jackson'

In [230]:
for name in directors.index:    
    directors[name] = data['revenue'][data['director'].map(lambda x : True if name in x else False)].sum()

directors.sort_values(ascending=False).head(1)

Peter Jackson    6490593685
dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [114]:
answers['14'] = 'Robert Rodriguez'

In [242]:
df = data[data['genres'].str.contains('Action')]['director']
split_values(df).head(1)

Robert Rodriguez    9
dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [115]:
answers['15'] = 'Chris Hemsworth'

In [182]:
df_2012 = data.query('release_year == 2012')
cast = split_values(df_2012['cast']).value_counts()

for name in cast.index:
    cast[name] = df_2012['revenue'][df_2012['cast'].str.contains(name)].sum()

cast.sort_values(ascending=False).head(1)

Chris Hemsworth    2027450773
dtype: int64

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [116]:
answers['16'] = 'Matt Damon '

In [183]:
split_values( data.query('budget > budget.mean()')['cast'] ) \
    .value_counts().head(1)

Matt Damon    18
dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [117]:
answers['17'] = 'Action'

In [237]:
genres_Cage = data[data['cast'].str.contains('Nicolas Cage')]['genres']
split_values(genres_Cage).head(1)

Action    17
dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [118]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'

In [20]:
data[data['production_companies'].str.contains('Paramount Pictures')] \
    .sort_values('profit')[['imdb_id', 'original_title']].head(1)

,imdb_id,original_title
925,tt0267626,K-19: The Widowmaker


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [119]:
answers['19'] = '2015'

In [61]:
data.groupby('release_year')['revenue'].agg('sum') \
    .sort_values(ascending=False).head(1)

release_year
2015    25449202382
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [120]:
answers['20'] = '2014'

In [45]:
df = data[data['production_companies'].str.contains('Warner Bros')]
df.groupby(['release_year'])['revenue'].sum().sort_values().tail(1)

release_year
2014    3243064519
Name: revenue, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [121]:
answers['21'] = 'Сентябрь'

In [49]:
data.groupby('release_month')['imdb_id'] \
    .agg('count').sort_values().tail(1)

release_month
September    227
Name: imdb_id, dtype: int64

Вариант 2

In [55]:
pd.pivot_table(data, index=['release_month'], values=['original_title'], aggfunc='count') \
    .sort_values('original_title', ascending=False).head(1)

,original_title
release_month,
September,227


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [122]:
answers['22'] = '450'

In [59]:
df = data.query('release_month == ("June", "July", "August")')
df.groupby('release_month')['imdb_id'].agg('count').sum()

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [123]:
answers['23'] = 'Peter Jackson'

In [81]:
df = data.query('release_month == ("December", "January", "February")')

for name in directors.index:    
    directors[name] = df['imdb_id'][df['director'].map(lambda x : True if name in x else False)].count()

directors.sort_values(ascending=False).head(1)

Peter Jackson    7
dtype: int64

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [124]:
answers['24'] = 'Four By Two Productions'

In [65]:
data.loc[:, 'title_length'] = data['original_title'].map(lambda x : len(x))
for name in companies.index:
    companies[name] = data['title_length'][data['production_companies'].map(lambda x : True if name in x else False)].mean()
companies.sort_values(ascending=False).head(1)

Four By Two Productions    83
dtype: int64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [125]:
answers['25'] = 'Midnight Picture Show'

In [68]:
data.loc[:, 'overview_words'] = data['overview'].map(lambda x : len(x.split()))
for name in companies.index:
    companies[name] = data['overview_words'][data['production_companies'].map(lambda x : True if name in x else False)].mean()
companies.sort_values(ascending=False).head(1)

Midnight Picture Show    175.0
dtype: float64

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [126]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

In [82]:
top = data['vote_average'].quantile(0.99)
data.query('vote_average > @top')[['original_title', 'vote_average']]


,original_title,vote_average
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
119,Guardians of the Galaxy,7.9
125,The Imitation Game,8.0
128,Gone Girl,7.9
138,The Grand Budapest Hotel,7.9
370,Inception,7.9
599,The Dark Knight,8.1
872,The Pianist,7.9


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [127]:
answers['27'] = 'Daniel Radcliffe & Rupert Grint'

In [82]:
cast = data['cast'].str.split("|")
cast_pairs = cast.map(lambda x : list(combinations(x,2))).sum()
pd.Series(cast_pairs).str.join(' & ')\
    .value_counts().sort_values(ascending=False).head(1)

Daniel Radcliffe & Rupert Grint    8
dtype: int64

ВАРИАНТ 2

In [83]:
def cast_pairs(x):
    lst = []
    for i in range(0, len(x)-1):
        for j in range(i+1, len(x)):
            lst.append(x[i] + " & " + x[j])
    return lst

cast = data['cast'].str.split('|')\
    .map(lambda x : cast_pairs(x)).sum()

sorted(Counter(cast).items(), key=lambda x:x[1], reverse=True)[:1]

[('Daniel Radcliffe & Rupert Grint', 8)]

# Submission

In [128]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650) ',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107',
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819) ',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon ',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'Сентябрь',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [129]:
len(answers)

27